# Jupyter Notebook for Reading & Displaying a GeoPackage

What is a GeoPackage?

Pythom Modules

Part of the power of the Python programming language is its extensibility through "importing" existing code into a program.  

sqlite3 - Allows a Python program to create a connection with a SQLite database, run SQL queries and return results.
pandas - Allows the easy manipulation of series and tabular data.
folium - provides a Python interface to Leaflet for the display of geospatial data within a Jupyter Notebook.
OGR/GDAL - provides processes for manipulating geospatial data. In this case it is used to create the geospatial indexes for any new tables created within the GeoPackage database


In [1]:
# Importing Modules
import sqlite3
import pandas
import os
import folium

Import OGR/GDAL. Sometimes OGR/GDAL are not available in some environments. This is an example of catching this situation.

In [3]:
try:
    import osgeo.gdal
    import osgeo.ogr
    # this makes sure exceptions within GDAL show
    osgeo.gdal.UseExceptions()
    osgeo.ogr.UseExceptions()
except Exception as e:
    _gdalAvailable = False
    print("Unable to load either the 'osgeo.gdal' or 'osgeo.ogr' module. You are probably not running this in a GDAL environment.")
    raw_input("\n\nPress ENTER to exit")
    raise

### Functions for ...

MAIN

Setting up some hard coded locations

In [4]:
appDir = '/Users/Marcus/SpatialData/Format/GeoPackage/ASGS/2011/'
geometryLayerJoinCol = "LGA_CODE_2011"

In [5]:
csvDataFile = os.path.join(appDir, "ABS_SEIFA_LGA_2011.txt")
csvDataJoinCol = "LGA_2011"

Creating a dataframe from a delimited file is enabled by the pandas.read_csv function. 

This is very flexible and whilst the deimited file doesn't specify the ASGS code as being a string data type we can force it with the dtype= parameter.

See Pandas Documentation for details.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

In [6]:
df_csv_seifa = pandas.read_csv(csvDataFile, sep=',', dtype={'LGA_CODE_2011': str} )
df_csv_seifa.head(10)

,LGA_CODE_2011,IRSAD_Score,IRSAD_Decile,IRSD_Score,IRSD_Decile,IER_Score,IER_Decile,IEO_Score,IEO_Decile,URP
0,10050,967,6,979,6,964,4,962,6,47851
1,10110,985,7,987,6,954,3,1034,9,24122
2,10150,1031,9,1015,8,962,3,1092,10,41216
3,10200,944,4,917,2,931,2,1001,8,73744
4,10250,980,7,989,6,986,6,985,7,39263
5,10300,929,3,946,3,969,4,930,3,2284
6,10350,946,4,932,3,961,3,966,6,182366
7,10470,985,7,991,7,994,7,977,7,38514
8,10550,951,5,969,5,974,5,952,5,31951
9,10600,942,4,950,4,957,3,980,7,12524


In [7]:
df_csv_seifa.tail(10)

,LGA_CODE_2011,IRSAD_Score,IRSAD_Decile,IRSD_Score,IRSD_Decile,IER_Score,IER_Decile,IEO_Score,IEO_Decile,URP
554,72300,1032,9,1030,9,1071,10,975,7,19000
555,72330,674,1,592,1,625,1,903,1,5848
556,72800,1023,9,1018,8,1015,8,982,7,27714
557,73600,650,1,578,1,588,1,854,1,6121
558,74050,667,1,602,1,589,1,908,2,2578
559,74500,662,1,581,1,613,1,877,1,5917
560,74560,977,6,984,6,1011,8,1007,8,368
561,74660,715,1,659,1,640,1,880,1,6224
562,79399,1016,8,1039,9,966,4,1005,8,7982
563,89399,1090,10,1076,10,1051,10,1115,10,356527


In [9]:
df_csv_seifa.shape

(564, 10)

In [10]:
df_csv_seifa.columns

Index(['LGA_CODE_2011', 'IRSAD_Score', 'IRSAD_Decile', 'IRSD_Score',
       'IRSD_Decile', 'IER_Score', 'IER_Decile', 'IEO_Score', 'IEO_Decile',
       'URP'],
      dtype='object')

In [11]:
df_csv_seifa.dtypes

LGA_CODE_2011    object
IRSAD_Score       int64
IRSAD_Decile      int64
IRSD_Score        int64
IRSD_Decile       int64
IER_Score         int64
IER_Decile        int64
IEO_Score         int64
IEO_Decile        int64
URP               int64
dtype: object

In [12]:
len(df_csv_seifa)

564

Loading the SEIFA CSV file into a GeoPackage

In [13]:
gpkgSourceFile = os.path.join(appDir, "ASGS_2011_LGA.gpkg")
gpkgSEIFATableName = 'LGA_2011_SEIFA'
con = sqlite3.connect(gpkgSourceFile)

df_csv_seifa.to_sql(gpkgSEIFATableName, con, if_exists='append', index=False)
df_gpkg_seifa = pandas.read_sql("SELECT * from " + gpkgSEIFATableName, con)

df_gpkg_seifa.head(10)


,LGA_CODE_2011,IRSAD_Score,IRSAD_Decile,IRSD_Score,IRSD_Decile,IER_Score,IER_Decile,IEO_Score,IEO_Decile,URP
0,10050,967,6,979,6,964,4,962,6,47851
1,10110,985,7,987,6,954,3,1034,9,24122
2,10150,1031,9,1015,8,962,3,1092,10,41216
3,10200,944,4,917,2,931,2,1001,8,73744
4,10250,980,7,989,6,986,6,985,7,39263
5,10300,929,3,946,3,969,4,930,3,2284
6,10350,946,4,932,3,961,3,966,6,182366
7,10470,985,7,991,7,994,7,977,7,38514
8,10550,951,5,969,5,974,5,952,5,31951
9,10600,942,4,950,4,957,3,980,7,12524


## Working with the GeoPackage

GeoPackage Schema

[Image of the GeoPackage schema]

In [14]:
gpkgSourceFile = os.path.join(appDir, "ASGS_2011_LGA.gpkg")

In [15]:
con = sqlite3.connect(gpkgSourceFile)
df_gpkg_contents = pandas.read_sql("SELECT * from  gpkg_contents", con)
df_gpkg_contents

,table_name,data_type,identifier,description,last_change,min_x,min_y,max_x,max_y,srs_id
0,ASGS_2011_LGA,features,ASGS_2011_LGA,,2018-03-03T07:27:03.497Z,0.0,-43.7405,159.109,0.0,4283


In [16]:
df_geom_table = pandas.read_sql("SELECT * from " + df_gpkg_contents.iloc[0,0], con)
df_geom_table.head(10)

,fid,geometry,OBJECTID,LGA_CODE_2011,LGA_NAME_2011,STE_CODE_2011,STE_NAME_2011
0,1,b'GP\x00\x03\xbb\x10\x00\x00\xa0\x04R<\x10Zb@8...,1,10050,Albury (C),1,New South Wales
1,2,b'GP\x00\x03\xbb\x10\x00\x00\x84\xe4\xc6\xc2J\...,2,10110,Armidale Dumaresq (A),1,New South Wales
2,3,b'GP\x00\x03\xbb\x10\x00\x00\xcc\x85(1\x98\xe3...,3,10150,Ashfield (A),1,New South Wales
3,4,b'GP\x00\x03\xbb\x10\x00\x00H\xd8-Xc\xe0b@ \x0...,4,10200,Auburn (C),1,New South Wales
4,5,b'GP\x00\x03\xbb\x10\x00\x00\xdc\xb8\x86\x98\x...,5,10250,Ballina (A),1,New South Wales
5,6,b'GP\x00\x03\xbb\x10\x00\x00\x88M*;y\xcea@\x88...,6,10300,Balranald (A),1,New South Wales
6,7,b'GP\x00\x03\xbb\x10\x00\x00t;\t\xac\xe9\xdeb@...,7,10350,Bankstown (C),1,New South Wales
7,8,b'GP\x00\x03\xbb\x10\x00\x00<\x88\x03\x0c\xbe\...,8,10470,Bathurst Regional (A),1,New South Wales
8,9,b'GP\x00\x03\xbb\x10\x00\x00\\\xbaW\xedg\xabb@...,9,10550,Bega Valley (A),1,New South Wales
9,10,b'GP\x00\x03\xbb\x10\x00\x00x\xa6O\xf9w\x0cc@\...,10,10600,Bellingen (A),1,New South Wales


In [17]:
df_geom_table.columns

Index(['fid', 'geometry', 'OBJECTID', 'LGA_CODE_2011', 'LGA_NAME_2011',
       'STE_CODE_2011', 'STE_NAME_2011'],
      dtype='object')

SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate
FROM Orders
INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;

In [18]:
sql = "CREATE TABLE AUS_LGA_2011_SEIFA AS " + "SELECT ASGS_2011_LGA.LGA_CODE_2011, ASGS_2011_LGA.geometry, IRSAD_Score, \
    IRSAD_Decile, IRSD_Score, IRSD_Decile, IER_Score, IER_Decile, IEO_Score, IEO_Decile \
    FROM ASGS_2011_LGA INNER JOIN LGA_2011_SEIFA ON LGA_2011_SEIFA.LGA_CODE_2011 = ASGS_2011_LGA.LGA_CODE_2011"
    
con.execute(sql)

con.close()

OperationalError: table AUS_LGA_2011_SEIFA already exists

In [19]:
"""Uses GDAL to create the indexes"""
try:
    gpkgDriver = osgeo.ogr.GetDriverByName("GPKG")
    conToDbFile = gpkgDriver.Open(gpkgSourceFile, 1)
    sql = "SELECT CreateSpatialIndex('AUS_LGA_2011_SEIFA', 'geometry')"
    rsHandle = conToDbFile.ExecuteSQL(sql)
    conToDbFile.ReleaseResultSet(rsHandle)
except Exception as e:
    #print("Error while spatial indexing column geometry for layer " + str(
    #    layername) + " in database '" + str(dbname) + "':\n" + str(e))
    raise


RuntimeError: Unknown geometry column name